In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
from time import time
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
import transformers
from transformers import AutoTokenizer,TFDistilBertModel, DistilBertConfig
from transformers import TFAutoModel

In [2]:
!pip install tensorflow

In [3]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install transformers

In [5]:
released = pd.read_excel(r"C:\Users\PShukla\Downloads\VW_CTDI_OrderHolds_Released_22_02_2023latest.xlsx")

In [6]:
released.head()

,ctdiOrderHoldsId,holdType,serviceLine,serviceRequest,partNBR,customer,systemEngineer,daysOnHold,holdCode,holdIssue,resolutionCode,resolutionResponse,startTime,endTime,insertDateTime,fileName,createdBy
0,2,PROCUREMENT,ATTATMIS+,IP22067957,XDC1001U,CISCO SYSTEMS,DL-InventoryMgt@list.att.com,8,MFG Part # Missing,XDC1001U - QTY.1 NO PART AVAILABLE,MFG PART # PROVIDED,part provided,2022-08-11T12:52:06-05:00,2022-08-19T12:36:40-05:00,"2/12/2023, 10:58:53 AM",ctdi_order_holds_08192022130343_decrypt.csv,EC2AMAZ-T2KLKUH\Administrator
1,3,PROCUREMENT,ATTATMIS+,R010025352,XDCLHSMD,NaN,DL-InventoryMgt@list.att.com,11,MFG Part # Missing,XDCLHSMD_x000d_\nQTY 1_x000d_\n_x000d_\nPLEASE...,MFG PART # PROVIDED,PART PROVIDED,2022-08-01T10:47:30-05:00,2022-08-12T14:04:53-05:00,"2/12/2023, 10:58:53 AM",ctdi_order_holds_08122022150353_decrypt.csv,EC2AMAZ-T2KLKUH\Administrator
2,4,PROCUREMENT,ATTMISMTCE,R010025332,GLC-TE,CISCO SYSTEMS,DL-InventoryMgt@list.att.com,13,MFG Part # Missing,GLC-TE_x000d_\nQTY 1_x000d_\n_x000d_\nPLEASE P...,MFG PART # PROVIDED,PART PROVIDED,2022-07-28T13:57:47-05:00,2022-08-10T13:38:23-05:00,"2/12/2023, 10:58:53 AM",ctdi_order_holds_08102022140403_decrypt.csv,EC2AMAZ-T2KLKUH\Administrator
3,5,PROCUREMENT,ATTSIGEU,AVTS220715111126,ISE1075,Custom Cable,DL-InventoryMgt@list.att.com,3,MFG Part # Missing,ISE1079_x000d_\nISE1076_x000d_\nISE1075_x000d_...,MFG PART # PROVIDED,"please use from these below from NCRTRANSFER, ...",2022-07-29T13:05:30-05:00,2022-08-01T13:08:29-05:00,"2/12/2023, 10:58:53 AM",ctdi_order_holds_08012022140403_decrypt.csv,EC2AMAZ-T2KLKUH\Administrator
4,6,PROCUREMENT,ATTSIGEU,AVTS220715111126,ISE1076,Custom Cable,DL-InventoryMgt@list.att.com,3,MFG Part # Missing,ISE1079_x000d_\nISE1076_x000d_\nISE1075_x000d_...,MFG PART # PROVIDED,"please use from these below from NCRTRANSFER, ...",2022-07-29T13:05:30-05:00,2022-08-01T13:08:29-05:00,"2/12/2023, 10:58:53 AM",ctdi_order_holds_08012022140403_decrypt.csv,EC2AMAZ-T2KLKUH\Administrator


In [7]:
duplicated= released[released.duplicated()]
duplicated.shape

(0, 17)

In [8]:
duplicated.head()

,ctdiOrderHoldsId,holdType,serviceLine,serviceRequest,partNBR,customer,systemEngineer,daysOnHold,holdCode,holdIssue,resolutionCode,resolutionResponse,startTime,endTime,insertDateTime,fileName,createdBy


In [9]:
released.columns

Index(['ctdiOrderHoldsId', 'holdType', 'serviceLine', 'serviceRequest',
       'partNBR', 'customer', 'systemEngineer', 'daysOnHold', 'holdCode',
       'holdIssue', 'resolutionCode', 'resolutionResponse', 'startTime',
       'endTime', 'insertDateTime', 'fileName', 'createdBy'],
      dtype='object')

In [10]:
final_df=released[['holdCode','holdIssue','resolutionCode']]

In [11]:
final_df['resolutionCode'].unique()

array(['MFG PART # PROVIDED', 'MFG PRT# UPDATED', 'ACK PROVIDED',
       'PO PROVIDED', 'HOLD IGNORE', 'MFG PART #  DELIVERED',
       'HOST NAME ERROR CLEARED', 'UPDATED EOL PROVIDED',
       'CFG FILE PROVIDED', 'UPDATED CFG FILE', 'UPDATED SDP FILE',
       'RCD REVISED CI', 'OTHER', 'SDP & CFG FILE PROVIDED',
       'IOS PROVIDED', 'HOST NAME PROVIDED', 'SDP FILE PROVIDED',
       'UPDATED IOS', 'UPDATED SDP&CFG FILE', 'ORDER CANCELLED',
       'ACK UPDATED', 'MORE INFORMATION NEEDED'], dtype=object)

In [12]:
# remove records which does not having any resolution code
final_df=final_df[final_df['resolutionCode'].isnull()==False]

In [13]:
final_df.shape

(1691, 3)

In [14]:
final_df2=final_df.reset_index (drop=True)

In [15]:
final_df2["holdCode_holdIssue"]=final_df2["holdCode"]+final_df2["holdIssue"]

In [16]:
final_df2['resolutionCode'].value_counts()

RCD REVISED CI             476
MFG PART # PROVIDED        329
UPDATED CFG FILE           181
OTHER                      162
ACK PROVIDED               132
SDP & CFG FILE PROVIDED    124
CFG FILE PROVIDED           55
UPDATED EOL PROVIDED        41
UPDATED SDP FILE            33
SDP FILE PROVIDED           26
UPDATED SDP&CFG FILE        25
IOS PROVIDED                21
ACK UPDATED                 16
PO PROVIDED                 12
HOST NAME ERROR CLEARED     12
ORDER CANCELLED             11
MORE INFORMATION NEEDED     10
UPDATED IOS                  9
HOST NAME PROVIDED           6
MFG PART #  DELIVERED        5
MFG PRT# UPDATED             4
HOLD IGNORE                  1
Name: resolutionCode, dtype: int64

In [17]:
def cleanText(words):
    words = re.sub("[^a-zA-Z]"," ",words)
    text = words.lower().split()
    return " ".join(text)

final_df2['holdCode_holdIssue'] = final_df2['holdCode_holdIssue'].apply(cleanText)

In [18]:
!pip install nltk
import nltk

In [19]:
stop = stopwords.words('english')
lem = WordNetLemmatizer()


def remove_stopwords(text):
    """The function to removing stopwords"""
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

def word_lem(text):
    """The function to apply lemmatizing"""
    lem_text = [lem.lemmatize(word) for word in text.split()]
    return " ".join(lem_text)

In [20]:
final_df2['holdCode_holdIssue'] = final_df2['holdCode_holdIssue'].apply(remove_stopwords)
final_df2['holdCode_holdIssue'] = final_df2['holdCode_holdIssue'].apply(word_lem)

In [35]:
#Voting
# Import the necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import ElasticNet
import xgboost
from xgboost import XGBClassifier

# Create the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('voting', VotingClassifier([
        ('lr', LogisticRegression()),
        ('svm', LinearSVC()),
        ('rf', RandomForestClassifier()),
        ('dtc', DecisionTreeClassifier()),
    
        
    ]))
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(final_df2['holdCode_holdIssue'], final_df2['resolutionCode'], test_size=0.2, shuffle=True)

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
accuracy = pipeline.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))
# Use trained model to make recommendations with multiple filters


Accuracy: 90.56%


In [36]:
recommendation = []
n=input("Enter the combination of hold code & hold issue:")

for i in (final_df2['holdCode_holdIssue'].unique()):
   
 if n[:3] == i[:3]:
     print(i)
    
     def make_recommendations(m,Accuracy_filter=None):
      text_vec = vectorizer.transform([m])
      pred = clf.predict(text_vec)
      recommendation.append(pred)

# Example usage with multiple filters
#text = "MFG"
m=input("Enter the filtered(specific) combination of hold code & hold issue:")
Accuracy_filter = 0.8
make_recommendations(m, Accuracy_filter=Accuracy_filter)
print('Recommendations:', recommendation)

Enter the combination of hold code & hold issue:cfg
cfg errorneed static ip information
cfg errorstatic ip information error
cfg errorrouter conf net host network configuration file host command replaced command copy url system running config address name remote host source filename router confg work ip cfg configure using tftp work ip cfg confirm loading work ip cfg via gigabitethernet ok byte boot system flash bootflash isr universalk spa bin invalid input detected marker ip tftp source gigabitethernet invalid input detected marker
cfg errorwrong config file x order xdc u router
cfg errorcard type x x invalid input detected marker x x network clock input source controller x x invalid input detected marker x x isdn switch type primary ni x x invalid input detected marker x warning codec built vad cannot disabled x codec negotiated vad configuration disregarded x x controller x x invalid input detected marker x x clock source line x x invalid input detected marker x x framing esf x x i